In [7]:
from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
    identity_with_bool_success,
)
from multicall import Call

from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)

from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama

from mainnet_launch.constants import (
    WORKING_DATA_DIR,
    AutopoolConstants,
    ChainData,
    AUTO_ETH,
    BASE_ETH,
    AUTO_LRT,
    BAL_ETH,
    DINERO_ETH,
)
from mainnet_launch.data_fetching.internal.s3_helper import fetch_rebalance_plan_json_no_s3_client
from mainnet_launch.pages.autopool.autopool_diagnostics.lens_contract import build_proxyGetDestinationSummaryStats_call

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

import json
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio


pio.templates.default = None




class PlanVerificationError(Exception):
    pass


def _validate_plan(plan_data: dict):
    """Raise a error if the plan is malformed"""
    if plan_data["destinationOut"] is None:
        raise PlanVerificationError("destinationOut is None")
    if plan_data["destinationIn"] is None:
        raise PlanVerificationError("destinationIn is None")


def add_destination_summary_stats(plan_file_path: str, rebalance_block: int, autopool):
    plan_data = fetch_rebalance_plan_json_no_s3_client(plan_file_path, autopool)
    _validate_plan(plan_data)
    destination_out_summary_stats_call = build_proxyGetDestinationSummaryStats_call(
        "out", autopool, plan_data["destinationOut"], direction="out", amount=0
    )
    destination_in_summary_stats_call = build_proxyGetDestinationSummaryStats_call(
        "in", autopool, plan_data["destinationIn"], direction="in", amount=0
    )

    stats_at_block = get_state_by_one_block(
        [destination_out_summary_stats_call, destination_in_summary_stats_call], rebalance_block, autopool.chain
    )

    stats_at_block['out']['virtual_price'] = None # stub
    stats_at_block['in']['virtual_price'] = None # stub

    plan_data['destinationOutSummaryStats'] = stats_at_block
    
    return plan_data


def fetch_and_augment_onchain_calc_plans(autopool: AutopoolConstants) -> dict:
    AUGMENTED_PLANS_SAVE_DIR = WORKING_DATA_DIR / f'{autopool.name}_augmented_plans'
    os.makedirs(AUGMENTED_PLANS_SAVE_DIR, exist_ok=True)

    # plans = get_full_table_as_df(
    #     RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr
    # )
    events = get_full_table_as_df_with_tx_hash(
        RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == autopool.autopool_eth_addr
    )
    # mainnet_blocks = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == autopool.chain.chain_id).sort_values(
    #     "block"
    # )
    # destinations = set(events["destination_out"].unique().tolist() + plans["destination_in"].unique().tolist())
    # destinations = {str(d) for d in destinations if pd.notna(d)}

    def fetch_and_save_augmented_plans(row):
        file_path = row['rebalance_file_path']
        block = row['block']
        augmented_plan = add_destination_summary_stats(file_path, block, autopool)

        with open(AUGMENTED_PLANS_SAVE_DIR / f"{augmented_plan['rebalance_plan_json_key']}.json", "w") as f:
            json.dump(augmented_plan, f, indent=4)
        # print(f"Saved augmented plan to {AUGMENTED_PLANS_SAVE_DIR / f'{file_path}.json'}")


    # Process all rows with rebalance_file_path using ThreadPoolExecutor
    filtered_events = events[events['rebalance_file_path'].notna()]
    # Print autopool name and count of events without plans
    events_without_plans = events[events['rebalance_file_path'].isna()]
    print(f"Autopool: {autopool.name}")
    print(f"Number of events without rebalance plans: {len(events_without_plans)}")

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_and_save_augmented_plans, row) for idx, row in filtered_events.iterrows()]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Augmenting and saving plans"):
            try:
                future.result()
            except Exception as e:
                print(f"Error processing plan: {e}")

    print(f"Processed {len(filtered_events)} events")


fetch_and_augment_onchain_calc_plans(AUTO_ETH)
fetch_and_augment_onchain_calc_plans(BASE_ETH)
fetch_and_augment_onchain_calc_plans(BAL_ETH)
fetch_and_augment_onchain_calc_plans(AUTO_LRT)
fetch_and_augment_onchain_calc_plans(DINERO_ETH)

Autopool: autoETH
Number of events without rebalance plans: 9


Augmenting and saving plans:   0%|          | 0/643 [00:00<?, ?it/s]

Processed 643 events
Autopool: baseETH
Number of events without rebalance plans: 0


Augmenting and saving plans:   0%|          | 0/265 [00:00<?, ?it/s]

Processed 265 events
Autopool: balETH
Number of events without rebalance plans: 3


...

Processed 295 events
Autopool: autoLRT
Number of events without rebalance plans: 2


Augmenting and saving plans:   0%|          | 0/241 [00:00<?, ?it/s]

Processed 241 events
Autopool: dineroETH
Number of events without rebalance plans: 1


Augmenting and saving plans:   0%|          | 0/76 [00:00<?, ?it/s]

Processed 76 events


In [ ]:
# next step is to get a destination vault address -> what ever the sovler use sfor the virtual price exchange rate type call

In [2]:
# import json

# calls = [Call(address, "getStats()(address)", [(address, identity_with_bool_success)]) for address in destinations]

# destination_vault_address_to_stats_contract = get_state_by_one_block(
#     calls, autopool.chain.get_block_near_top(), autopool.chain
# )

# calls = [
#     Call(stats_contract, "underlyerStats()(address)", [(destination_vault, identity_with_bool_success)])
#     for destination_vault, stats_contract in destination_vault_address_to_stats_contract.items()
#     if stats_contract is not None
# ]

# destination_vault_to_underlyer = get_state_by_one_block(calls, autopool.chain.get_block_near_top(), autopool.chain)

# calls = [
#     Call(stats_contract, "pool()(address)", [(destination_vault, identity_with_bool_success)])
#     for destination_vault, stats_contract in destination_vault_address_to_stats_contract.items()
#     if stats_contract is not None
# ]
# virtual_price_call = "    function get_virtual_price() external view returns (uint256);"
# destination_vault_to_pool = get_state_by_one_block(calls, autopool.chain.get_block_near_top(), autopool.chain)

# I don't  think we're using this part

In [3]:
# os.makedirs(AUGMENTED_PLANS_SAVE_DIR, exist_ok=True)
# # Process first 10 rows sequentially for testing
# for idx, row in events[events['rebalance_file_path'].notna()].head(10).iterrows():
#     augmented_plan = fetch_and_save_augmented_plans(row)






# older comments and notes

In [4]:
# {'snapshotTimestamp': 0,
#  'name': 'Tokemak-Wrapped Ether-Instadapp ETH v2',
#  'address': '0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7',
#  'poolType': 'self',
#  'pool': '0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78',
#  'underlying': '0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78',
#  'underlyingTokens': ['0xA0D3707c569ff8C87FA923d3823eC5D81c98Be78'],
#  'underlyingTokenAmounts': [0],
#  'underlyingReserves': [],
#  'totalAprIn': 0.04768909754724683,
#  'totalAprOut': 0.04778744504724683,
#  'incentiveAPR': 0.0,
#  'destLPValue': 0,
#  'discountViolationAddFlag': [False],
#  'discountViolationTrim1Flag': [False],
#  'discountViolationTrim2Flag': [False],
#  'ownedShares': 876435544936734498733,
#  'totSupply': 0,
#  'safePrice': 1.199678846454828,
#  'spotPrice': 1.199706024959943,
#  'tokenSpotPrice': [1.199706024959943],# why is there a .3bps sptead between the safe and spot price? I thought it was like aave
#  'tokenSafePrice': [1.199678846454828],
#  'flipFlag': False},



# these are too old, not current
# def build_last_snapshot_timestamp_call(destination_vault_address: str, underlyer_address: str):
#     """Build a call to get lastSnapshotTimestamp from underlyer"""
#     return Call(
#         underlyer_address,
#         "lastSnapshotTimestamp()(uint256)",
#         [(f"{destination_vault_address}_timestamp", identity_with_bool_success)]
#     )

# def build_last_virtual_price_call(destination_vault_address: str, underlyer_address: str):
#     """Build a call to get lastVirtualPrice from underlyer"""
#     return Call(
#         underlyer_address,
#         "lastVirtualPrice()(uint256)",
#         [(f"{destination_vault_address}_vp", safe_normalize_with_bool_success)]
#     )

# all_timestamp_calls = [
#     build_last_snapshot_timestamp_call(destination_vault_address, underlyer_address)
#     for destination_vault_address, underlyer_address in destination_vault_to_underlyer.items()
#     if underlyer_address is not None
# ]

# all_last_vp_calls = [
#     build_last_virtual_price_call(destination_vault_address, underlyer_address)
#     for destination_vault_address, underlyer_address in destination_vault_to_underlyer.items()
#     if underlyer_address is not None
# ]

# all_virtual_price_calls = all_timestamp_calls + all_last_vp_calls

# vps = get_state_by_one_block(all_virtual_price_calls, autopool.chain.get_block_near_top(), autopool.chain)
# vps

In [5]:
# not working
# valid_pools = {dest: pool for dest, pool in destination_vault_to_pool.items() if pool is not None}

# # Build convertToAssets calls for each pool
# convert_to_assets_calls = [
#     Call(
#         pool_address,
#         ["convertToAssets(uint256)(uint256)", int(1e18)],
#         [(f"{destination_vault}_assets", safe_normalize_with_bool_success)]
#     )
#     for destination_vault, pool_address in valid_pools.items()
# ]
 #broke
# # Execute the calls
# pool_assets = get_state_by_one_block(convert_to_assets_calls, autopool.chain.get_block_near_top(), autopool.chain)
# pool_assets

In [6]:
plan_data

NameError: name 'plan_data' is not defined